In [ ]:
# ---------------------------------------------------------------------------- #
# An implementation of https://arxiv.org/pdf/1512.03385.pdf                    #
# See section 4.2 for the model architecture on CIFAR-10                       #
# Some part of the code was referenced from below                              #
# https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py   #
# ---------------------------------------------------------------------------- #

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
import torch.utils.data as data
import numpy as np


# Device configuration
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 100
learning_rate = 0.001
batch_size = 100

# Image preprocessing modules
CIFAR_MEAN = [0.49139968, 0.48215827, 0.44653124]
CIFAR_STD = [0.24703233, 0.24348505, 0.26158768]

train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
])

class CIFAR10_truncated(data.Dataset):

    def __init__(self, root, dataidxs=None, train=True, transform=None, target_transform=None, download=False):

        self.root = root
        self.dataidxs = dataidxs
        self.train = train
        self.transform = transform
        self.target_transform = target_transform
        self.download = download

        self.data, self.target = self.__build_truncated_dataset__()

    def __build_truncated_dataset__(self):
        print("download = " + str(self.download))
        cifar_dataobj = CIFAR10(self.root, self.train, self.transform, self.target_transform, self.download)

        if self.train:
            # print("train member of the class: {}".format(self.train))
            # data = cifar_dataobj.train_data
            data = cifar_dataobj.data
            target = np.array(cifar_dataobj.targets)
        else:
            data = cifar_dataobj.data
            target = np.array(cifar_dataobj.targets)

        if self.dataidxs is not None:
            data = data[self.dataidxs]
            target = target[self.dataidxs]

        return data, target

    def truncate_channel(self, index):
        for i in range(index.shape[0]):
            gs_index = index[i]
            self.data[gs_index, :, :, 1] = 0.0
            self.data[gs_index, :, :, 2] = 0.0

    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        img, target = self.data[index], self.target[index]

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        return len(self.data)

class Cutout(object):
    def __init__(self, length):
        self.length = length

    def __call__(self, img):
        h, w = img.size(1), img.size(2)
        mask = np.ones((h, w), np.float32)
        y = np.random.randint(h)
        x = np.random.randint(w)

        y1 = np.clip(y - self.length // 2, 0, h)
        y2 = np.clip(y + self.length // 2, 0, h)
        x1 = np.clip(x - self.length // 2, 0, w)
        x2 = np.clip(x + self.length // 2, 0, w)

        mask[y1: y2, x1: x2] = 0.
        mask = torch.from_numpy(mask)
        mask = mask.expand_as(img)
        img *= mask
        return img

train_transform.transforms.append(Cutout(16))

valid_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
    ])

# CIFAR-10 dataset
# train_dataset = torchvision.datasets.CIFAR10(root='./../../../data/cifar10',
#                                              train=True, 
#                                              transform=train_transform,
#                                              download=True)

# test_dataset = torchvision.datasets.CIFAR10(root='./../../../data/cifar10',
#                                             train=False, 
#                                             transform=valid_transform)
dl_obj = CIFAR10_truncated
dataidxs = np.random.randint(0, 50000, (5000,), dtype='int64')
train_ds = dl_obj('./../../../data/cifar10', dataidxs=dataidxs, train=True, transform=train_transform, download=True)
test_ds = dl_obj('./../../../data/cifar10', train=False, transform=valid_transform, download=True)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_ds,
                                           batch_size=100, 
                                           shuffle=True, 
                                           drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=test_ds,
                                          batch_size=100, 
                                          shuffle=False)

# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

model = ResNet(ResidualBlock, [6, 6, 6]).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'resnet.ckpt')

download = True
Files already downloaded and verified
download = True
Files already downloaded and verified
Epoch [1/100], Step [10/50] Loss: 2.3097
Epoch [1/100], Step [20/50] Loss: 2.1513
Epoch [1/100], Step [30/50] Loss: 1.9409
Epoch [1/100], Step [40/50] Loss: 2.0042
Epoch [1/100], Step [50/50] Loss: 1.8535
Epoch [2/100], Step [10/50] Loss: 1.8577
Epoch [2/100], Step [20/50] Loss: 1.7061
Epoch [2/100], Step [30/50] Loss: 1.8455
Epoch [2/100], Step [40/50] Loss: 1.8711
Epoch [2/100], Step [50/50] Loss: 1.7461
Epoch [3/100], Step [10/50] Loss: 1.6468
Epoch [3/100], Step [20/50] Loss: 1.6990
Epoch [3/100], Step [30/50] Loss: 1.7162
Epoch [3/100], Step [40/50] Loss: 1.7386
Epoch [3/100], Step [50/50] Loss: 1.5678
Epoch [4/100], Step [10/50] Loss: 1.7258
Epoch [4/100], Step [20/50] Loss: 1.6700
Epoch [4/100], Step [30/50] Loss: 1.6423
Epoch [4/100], Step [40/50] Loss: 1.8098
Epoch [4/100], Step [50/50] Loss: 1.6562
Epoch [5/100], Step [10/50] Loss: 1.7315
Epoch [5/100], Step [20/50] Los

Epoch [39/100], Step [50/50] Loss: 0.7229
Epoch [40/100], Step [10/50] Loss: 0.7291
Epoch [40/100], Step [20/50] Loss: 0.7540
Epoch [40/100], Step [30/50] Loss: 0.6371
Epoch [40/100], Step [40/50] Loss: 0.8487
Epoch [40/100], Step [50/50] Loss: 0.8123
Epoch [41/100], Step [10/50] Loss: 0.7324
Epoch [41/100], Step [20/50] Loss: 0.9055
Epoch [41/100], Step [30/50] Loss: 0.6436
Epoch [41/100], Step [40/50] Loss: 0.7270
Epoch [41/100], Step [50/50] Loss: 0.8121
Epoch [42/100], Step [10/50] Loss: 0.6432
Epoch [42/100], Step [20/50] Loss: 0.5229
Epoch [42/100], Step [30/50] Loss: 0.6942
Epoch [42/100], Step [40/50] Loss: 0.6794
Epoch [42/100], Step [50/50] Loss: 0.8356
Epoch [43/100], Step [10/50] Loss: 0.5833
Epoch [43/100], Step [20/50] Loss: 0.7923
Epoch [43/100], Step [30/50] Loss: 0.7608
Epoch [43/100], Step [40/50] Loss: 0.8459
Epoch [43/100], Step [50/50] Loss: 0.7817
Epoch [44/100], Step [10/50] Loss: 0.6849
Epoch [44/100], Step [20/50] Loss: 0.7049
Epoch [44/100], Step [30/50] Loss: